In [1]:
import os
from pathlib import Path

NOTEBOOK_PATH: Path = Path(os.path.abspath(''))

DATA_PATH: Path = NOTEBOOK_PATH.parent.joinpath('data')

PATH_FOR_RESULT: Path = NOTEBOOK_PATH.joinpath('RESULT')

if not PATH_FOR_RESULT.exists():
    PATH_FOR_RESULT.mkdir()

print(f'当前运行目录：{NOTEBOOK_PATH}')
print(f'当前数据目录：{DATA_PATH}')

当前运行目录：D:\Development\Python\InvestmentWorkshop\notebook
当前数据目录：D:\Development\Python\InvestmentWorkshop\data


# 缠论 02C-01

合并K线，完整合约计算。使用 SHFE.hc2110 合约。

## 准备

### 引用模块

In [2]:
import pandas as pd

from utility import (
    get_available_datafile_name,
    load_csv_as_dataframe,
    get_saved_filename,
    save_dataframe_to_csv
)
from InvestmentWorkshop.indicator import (
    pbx,
    merge_candlestick_dataframe,
)
from InvestmentWorkshop.plot import (
    plot_merged_candlestick,
)

Load succeed.


### 设置 pandas

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)

### 可用数据

In [4]:
available_datafile = get_available_datafile_name(DATA_PATH, 'merged')
for symbol, _ in available_datafile:
    print(f'symbol: {symbol:>12}')

symbol:    DCE.c2201
symbol:    DCE.i2201
symbol:  SHFE.ag2106
symbol:  SHFE.al2111
symbol:  SHFE.al2112
symbol:  SHFE.hc2110
symbol:  SHFE.hc2201
symbol:  SHFE.hc2202
symbol:  SHFE.rb2201


### 声明数据

In [5]:
# 代码：
symbol: str = 'DCE.c2201'

## 加载并绘图

### 加载

In [6]:
# 加载数据
df_data: pd.DataFrame = load_csv_as_dataframe(DATA_PATH.joinpath(f'{symbol}_Minute.csv'))
df_merged: pd.DataFrame = load_csv_as_dataframe(DATA_PATH.joinpath(f'{symbol}_merged.csv'))

 # 丢弃不需要的 column。
# columns_origin = list(df_data.columns)
# columns_needed = ['open', 'high', 'low', 'close']
# columns_drop = [x for x in columns_origin if x not in columns_needed]
# print(columns_drop)
# df_data.drop(columns=columns_drop, inplace=True)

# 显示计算用数据
df_data.info()
print('-' * 60)
df_merged.info()
print('-' * 100)
df_data.head(20)
print('-' * 60)
df_merged.head(5)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 62535 entries, 2021-01-15 21:00:00 to 2021-10-22 22:59:00
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   datetime_nano  62535 non-null  int64  
 1   open           62535 non-null  float64
 2   high           62535 non-null  float64
 3   low            62535 non-null  float64
 4   close          62535 non-null  float64
 5   volume         62535 non-null  int64  
 6   open_oi        62535 non-null  int64  
 7   close_oi       62535 non-null  int64  
dtypes: float64(4), int64(4)
memory usage: 4.3 MB
------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 62535 entries, 2021-01-15 21:00:00 to 2021-10-22 22:59:00
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   包含？     62535 non-null  bool   
 1   本K更高？   62535 non-null  bool   
 2   本K更低？   62

,包含？,本K更高？,本K更低？,前K本K方向,合并后高点,合并后低点,合并后周期,笔
datetime,,,,,,,,
2021-01-15 21:00:00,False,False,False,无效,2741.0,2741.0,1,NaN
2021-01-15 21:01:00,True,False,True,包含,NaN,NaN,2,NaN
2021-01-15 21:02:00,False,False,False,NaN,2741.0,2741.0,1,NaN
2021-01-15 21:03:00,True,True,False,包含,NaN,NaN,2,NaN
2021-01-15 21:04:00,False,False,False,NaN,2742.0,2742.0,1,NaN


### 保存文件名

In [7]:
import datetime as dt

# 文件名模板
pattern = '{symbol}_{i}_of_{n}.png'

# 每张图多少根K线
count: int = 600
# 多少重复
duplicated: int = 60

# 第1张图，0 ~ 800
# 第2张图，800-60 ~ 800*2-60
# 第3张图，800*2-60 ~ 800*3-60

n: int = len(df_data) // count
if len(df_data) % n > 0:
    n += 1

time_start = dt.datetime.now()
for i in range(n):
    filename: Path = PATH_FOR_RESULT.joinpath(
        pattern.format(symbol=symbol, i=i+1, n=n)
    )
    start = count * i - duplicated if i > 0 else 0
    end = count * (i + 1) -duplicated if i > 0 else count
    print(f'开始绘制第 {i+1} / {n} 幅图片 ...')
    plot_merged_candlestick(
        df_price=df_data[start:end],
        df_merged=df_merged[start:end],
        file_path=filename,
        merged_line_width=2,
        x=6400,
        y=4800,
        dpi=288
    )
    print(f'图片已保存在 {filename} 。')

time_end = dt.datetime.now()
print(f'全部完成，总用时： {time_end - time_start}')

开始绘制第 1 / 105 幅图片 ...
图片已保存在 D:\Development\Python\InvestmentWorkshop\notebook\RESULT\DCE.c2201_1_of_105.png 。
开始绘制第 2 / 105 幅图片 ...
图片已保存在 D:\Development\Python\InvestmentWorkshop\notebook\RESULT\DCE.c2201_2_of_105.png 。
开始绘制第 3 / 105 幅图片 ...
图片已保存在 D:\Development\Python\InvestmentWorkshop\notebook\RESULT\DCE.c2201_3_of_105.png 。
开始绘制第 4 / 105 幅图片 ...
图片已保存在 D:\Development\Python\InvestmentWorkshop\notebook\RESULT\DCE.c2201_4_of_105.png 。
开始绘制第 5 / 105 幅图片 ...
图片已保存在 D:\Development\Python\InvestmentWorkshop\notebook\RESULT\DCE.c2201_5_of_105.png 。
开始绘制第 6 / 105 幅图片 ...
图片已保存在 D:\Development\Python\InvestmentWorkshop\notebook\RESULT\DCE.c2201_6_of_105.png 。
开始绘制第 7 / 105 幅图片 ...
图片已保存在 D:\Development\Python\InvestmentWorkshop\notebook\RESULT\DCE.c2201_7_of_105.png 。
开始绘制第 8 / 105 幅图片 ...
图片已保存在 D:\Development\Python\InvestmentWorkshop\notebook\RESULT\DCE.c2201_8_of_105.png 。
开始绘制第 9 / 105 幅图片 ...
图片已保存在 D:\Development\Python\InvestmentWorkshop\notebook\RESULT\DCE.c2201_9_of_105.png 。
开

### 绘图

In [8]:
# n=800
# plot_merged_candlestick(
#     df_price=df_data[:n].copy(),
#     df_merged=df_merged[:n].copy(),
#     file_path=filename,
#     merged_line_width=3,
#     x=32000,
#     y=18000,
#     dpi=72,
#     debug=True
# )